# 多头注意力



In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

选择缩放点积注意力作为每一个注意力头

In [2]:
'''
key_size：输入Key的原始维度
query_size：输入Query的原始维度
value_size：输入Value的原始维度
num_hiddens：投影后的统一维度（必须是num_heads的整数倍）
num_heads：注意力头数（如8头注意力）
dropout：Dropout比率
'''
class MultiHeadAttention(nn.Module):
    """多头注意力"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout) # 单个缩放点积注意力实例
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias) # 查询投影
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias) # 键投影
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias) # 值投影
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias) # 输出投影（合并多头结果）

    def forward(self, queries, keys, values, valid_lens):
        '''
        步骤1：投影到统一空间
        步骤2：分离头维度（核心操作）
        transpose_qkv 的作用：
        逻辑上将num_hiddens 拆分为num_heads个独立子空间,物理上通过重塑和转置，让8个头能在一次批量矩阵运算中并行计算
        '''
        # (batch,n_q,query_size)→(batch,n_q,num_hiddens)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        # (batch,n_k,key_size)→(batch,n_k,num_hiddens)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        # (batch,n_v,value_size)→(batch,n_v,num_hiddens)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        # 步骤3：处理有效长度
        # 每个头需要相同的有效长度信息。若valid_lens形状为(32,)，重复后变为(256,)，与变换后的batch*num_heads匹配
        if valid_lens is not None:
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)
        # 步骤4：并行执行注意力
        '''
        输入:(256,10,32)×(256,10,32)×(256,10,32)
        输出:(256,10,32)  # 每个头输出32维
        '''
        output = self.attention(queries, keys, values, valid_lens)
        '''
        步骤5：合并多头（逆变换）
        变换后:(256,10,32)→(32,10,256)  # 恢复原始批次维度
        transpose_output是transpose_qkv的逆操作，将8个头的输出拼接回完整的num_hiddens维度
        '''
        output_concat = transpose_output(output, self.num_heads)
        # 步骤6：最终输出投影:(32,10,256)→(32,10,256)
        # 通过可学习的线性层融合各头信息，增强表达能力。类似CNN中多个卷积核的输出融合
        return self.W_o(output_concat)

使多个头并行计算

| 参数维度             | 含义                                         |
| :--------------- | :----------------------------------------- |
| **`X.shape[0]`** | `batch_size`：批次中的样本数量                      |
| **`X.shape[1]`** | `seq_len`：序列长度（句子中的词元数）                    |
| **`X.shape[2]`** | `num_hiddens`：投影后的特征维度（必须是`num_heads`的整数倍） |


In [3]:
'''
transpose_qkv：为并行计算变换形状
目标：将(batch,seq_len,num_hiddens)转换为(batch*num_heads,seq_len,num_hiddens/num_heads)
'''
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    '''
    reshape(...,num_heads,-1):将num_hiddens拆分为num_heads个子空间;示例：6维拆分为2头×3维
    形状变化(2,3,6)→(2,3,2,3)
    假设X:(batch=2,seq_len=3,num_hiddens=6),num_heads=2
    结果:(2,3,2,3)  -1:自动计算为6/2=3
    '''
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    '''
    permute(0,2,1,3):交换维度顺序，将num_heads维度提前
    形状变化(2,3,2,3)→(2,2,3,3)
    逻辑意义：准备将batch和num_heads两个维度合并
    结果:(2,2,3,3)新维度顺序:batch,num_heads,seq_len,-1
    '''
    X = X.permute(0, 2, 1, 3)
    '''
    reshape(-1,X.shape[2],X.shape[3]):合并batch和num_heads维度，实现逻辑并行
    -1计算：2(batch)× 2(heads)=4
    形状变化：(2,2,3,3)→(4,3,3)
    关键：现在GPU认为在处理4个独立样本，实际是2个样本×2个头并行
    结果:(4,3,3)  # 逻辑上:(batch*num_heads,seq_len,num_hiddens/num_heads)
    '''
    return X.reshape(-1, X.shape[2], X.shape[3])

'''
transpose_output：逆转变换，恢复原始形状
目标：将(batch*num_heads,seq_len,num_hiddens/num_heads)转换回(batch,seq_len,num_hiddens)
'''
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    '''
    reshape(-1,num_heads,X.shape[1],X.shape[2]):分离合并的batch*num_heads维度
    -1计算：4/2=2
    形状变化(4,3,3)→(2,2,3,3)
    假设X:(batch*num_heads=4,seq_len=3,head_dim=3),num_heads=2
    结果:(2,2,3,3)  # 分离batch和num_heads维度
    '''
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    '''
    permute(0,2,1,3):恢复维度顺序（transpose_qkv中permute的逆操作）
    形状变化(2,2,3,3)→(2,3,2,3)
    结果:(2,3,2,3)  # 恢复原始维度顺序:batch,seq_len,num_heads,head_dim
    '''
    X = X.permute(0, 2, 1, 3)
    '''
    reshape(X.shape[0],X.shape[1],-1):合并num_heads和head_dim维度，恢复num_hiddens
    -1计算：2×3=6
    形状变化(2,3,2,3)→(2,3,6)
    结果:(2,3,6)  # 合并num_heads和head_dim
    '''
    return X.reshape(X.shape[0], X.shape[1], -1)

测试

In [4]:
'''
num_hiddens=100：投影后的统一特征维度（必须是num_heads的整数倍）
num_heads=5：并行注意力头的数量
关键关系：每个头分得100/5=20维的子空间
'''
num_hiddens, num_heads = 100, 5
'''
key_size/query_size/value_size：输入Q/K/V的原始维度（此处均为100）
num_hiddens：投影后的统一维度（100），也是最终输出维度
num_heads=5：5个头并行计算
dropout=0.5：注意力权重上的dropout比率（较高，防止过拟合）
内部创建的层：
W_q/W_k/W_v：各100→100维投影层
W_o：最终输出投影层100→100维
DotProductAttention：缩放点积注意力实例
'''
attention = MultiHeadAttention(num_hiddens, # key_size=100
                               num_hiddens, # query_size=100  
                               num_hiddens, # value_size=100
                               num_hiddens, # num_hiddens=100 （投影后维度）
                               num_heads, # 5个头
                               0.5) # dropout=0.5（训练时50%随机失活）
'''
关闭dropout层（推理时不需要随机失活）
关闭BatchNorm的动量更新（本例中未使用）
确保输出确定性：相同输入总是得到相同输出
训练 vs 评估：
训练模式：attention.train()（默认），dropout激活
评估模式：attention.eval()，dropout关闭
'''
attention.eval() # 设置为评估模式

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

调用过程与形状追踪<br>
1. 输入
```Python
X (Query): (2, 4, 100)  # 2个样本，4个查询，每查询100维
Y (Key):   (2, 6, 100)  # 2个样本，6个键，每键100维
Y (Value): (2, 6, 100)  # Value与Key同形状
valid_lens: torch.tensor([3, 2])  # 样本0只关注前3个键，样本1只关注前2个键
```
2. 多头注意力内部变换
```Python
# 投影层 W_q/W_k/W_v: (2, ?, 100) → (2, ?, 100)
# 分头 (5个头): (2, ?, 100) → (10, ?, 20)  # 2*5=10, 100/5=20

# 处理valid_lens:
valid_lens = torch.repeat_interleave(valid_lens, repeats=5, dim=0)
# 结果: [3,3,3,3,3, 2,2,2,2,2] 形状 (10,)
# 每个样本的valid_len被复制5次，匹配变换后的batch维度
```
3. 缩放点积注意力计算
```Python
# Query: (10, 4, 20) × Key.transpose: (10, 20, 6) → Scores: (10, 4, 6)
# 带掩蔽softmax后，无效位置置0
# Attention × Value: (10, 4, 6) × (10, 6, 20) → Output: (10, 4, 20)
```
4. 合并多头与输出投影
```Python
# 合头: (10, 4, 20) → (2, 4, 100)
# 输出投影 W_o: (2, 4, 100) → (2, 4, 100)
```
5. 最终输出形状
```Python
attention(X, Y, Y, valid_lens).shape  # 结果: torch.Size([2, 4, 100])
```

In [5]:
batch_size, num_queries = 2, 4 # 2个样本，每个样本4个查询
num_kvpairs, valid_lens =  6, torch.tensor([3, 2]) # 6个键值对，有效长度分别为3和2
X = torch.ones((batch_size, num_queries, num_hiddens)) # Query张量: (2,4,100)
Y = torch.ones((batch_size, num_kvpairs, num_hiddens)) # Key/Value张量: (2,6,100)
'''
batch_size=2：保留了原始批次大小
num_queries=4：输出序列长度与查询数量一致（每个查询生成一个输出向量）
num_hiddens=100：输出特征维度与投影后维度一致（也是最终输出维度）
'''
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])